# Understanding prim's implementation 
https://www.geeksforgeeks.org/prims-algorithm-using-priority_queue-stl/

## Adjacency list representation 

In [42]:
# Adjacency list is indexed by from_vertex
# it is a list of tuples in the form (to_vertex, weight)
# it has duplicated entries permuting from and to vertices
AdjLst = g.adj
print(AdjLst)
# It has this form but it more efficient to workwith if a little less intuitive
# v : [(u1, w(vu1)), (u2, w(vu2)), (u3, w(vu3))...]  ## Represent edges incident on v
{ v: nbrs for v, nbrs in enumerate(AdjLst) }



[[(1, 1), (7, 2)], [(0, 1), (2, 2), (7, 2)], [(1, 2), (3, 2), (8, 2), (5, 2)], [(2, 2), (4, 2), (5, 1)], [(3, 2), (5, 2)], [(2, 2), (3, 1), (4, 2), (6, 1)], [(5, 1), (7, 1), (8, 2)], [(0, 2), (1, 2), (6, 1), (8, 2)], [(2, 2), (6, 2), (7, 2)]]


{0: [(1, 1), (7, 2)],
 1: [(0, 1), (2, 2), (7, 2)],
 2: [(1, 2), (3, 2), (8, 2), (5, 2)],
 3: [(2, 2), (4, 2), (5, 1)],
 4: [(3, 2), (5, 2)],
 5: [(2, 2), (3, 1), (4, 2), (6, 1)],
 6: [(5, 1), (7, 1), (8, 2)],
 7: [(0, 2), (1, 2), (6, 1), (8, 2)],
 8: [(2, 2), (6, 2), (7, 2)]}

## Priority Queues 

In [16]:
pq = []
heapq.heappush(pq, (0, "A"))  # (priority, Obj)
heapq.heappush(pq, (-3, "A"))
heapq.heappush(pq, (1, "B"))
heapq.heappush(pq, (2, "C"))
heapq.heappush(pq, (3, "D"))
heapq.heappush(pq, (3, "D"))
print("Note how it is kept sorted in ascending order of priority:")
print(pq)
u = heapq.heappop(pq)
print(f"And when we pop from it we take the lowest priority elem first (left pop), here: {u}")
print(pq)



Note how it is kept sorted in ascending order of priority:
[(-3, 'A'), (0, 'A'), (1, 'B'), (2, 'C'), (3, 'D'), (3, 'D')]
And when we pop from it we take the lowest priority elem first (left pop), here: (-3, 'A')
[(0, 'A'), (2, 'C'), (1, 'B'), (3, 'D'), (3, 'D')]


# TEsts

In [258]:
import heapq

    # This class represents a directed graph using adjacency list representation
class Graph:
    def __init__(self, V):
        self.V = V
        self.adj = [[] for _ in range(V)]

    # Function to add an edge to the graph
    def add_edge(self, u, v, w):
        self.adj[u].append((v, w))
        self.adj[v].append((u, w))
        

    def maxConnectedOnes(self, k):
        pq = [] # Priority queue to store vertices that are being processed
        src = 0 # Taking vertex 0 as the source

        key = [float('inf')] * self.V
        parent = [-1] * self.V
        in_mst = [False] * self.V
        heapq.heappush(pq, (0, src))
        key[src] = 0 
        
        CURR_CONSECUTIVE_ONES = 0
        cocs = [] # priority queue to store sizes of connected-ones-clusters "COCs"
        
        while pq:
            u = heapq.heappop(pq)[1] 
    
            if in_mst[u]:
                continue

            in_mst[u] = True

            #####################################
            print(f"{parent[u]}->{u}, w(uv) = {key[u]}")
            # the last count doesn't get ADDED!!!! @TODO 
            # need to keep this stuff in a list?
            if key[u] == 1: 
                CURR_CONSECUTIVE_ONES += 1
            else:
                if CURR_CONSECUTIVE_ONES != 0:                        
                    heapq.heappush(cocs, CURR_CONSECUTIVE_ONES)
                    CURR_CONSECUTIVE_ONES = 0
#             print(f"{CURR_CONSECUTIVE_ONES}/{MAX_CONSECUTIVE_ONES}") 
#             print(f"w: {key[u]}, coc: {CURR_CONSECUTIVE_ONES}")

            if CURR_CONSECUTIVE_ONES >= k:
                return k

            #####################################
            for v, weight in self.adj[u]:
                if not in_mst[v] and key[v] > weight:
                    key[v] = weight
                    heapq.heappush(pq, (key[v], v))
                    parent[v] = u
        
        if CURR_CONSECUTIVE_ONES != 0:
            # about to end, wrap up the last block of 1s
            heapq.heappush(cocs, CURR_CONSECUTIVE_ONES)

        ANS = 0
        print(cocs)
        while cocs:
            block_len = heapq.heappop(cocs)
            print(f"k={k}, blocklen={block_len}")
            if block_len < k:
                # use all the ones.
                ANS += block_len 
                ANS += 2  # The constants are for the in between 2 edge
                k -= block_len
                k -=1
            else:
                # We're done. use as many as you 
                ANS += k
                k = 0
            print(f" ANS:{ANS}, new_k: {k}")
        if k != 0:
            ANS += 2*k # fill remaining with 2s
        return ANS
        
    


In [ ]:
def generate_graph(q_oneW_edges, V):
    # (1,2), (2,3) .... all of which have weoght one
    # just make sure you order them in input
    g = Graph(V)
    for u in range(V):
        for v in range(u+1, V):
            if q_oneW_edges != []:
                x,y = q_oneW_edges[0]
                if u==x and y==v:
                    g.add_edge(u,v, 1)
#                     print(f"{u},{v},1*")
                    q_oneW_edges.pop(0)
                else:
                    g.add_edge(u,v, 2)
#                     print(f"{u},{v},2")
            else:
                g.add_edge(u,v, 2)
#                 print(f"{u},{v},2")

#                 g.add_edge(u,v, 2)
    return g



In [156]:
def generate_input(q_oneW_edges, V, k):
    # (1,2), (2,3) .... all of which have weoght one
    # just make sure you order them in input
    with open("input.txt", "w") as file1:
        file1.write(f"{V} {k}\n")
        for u in range(V):
            for v in range(u+1, V):
                wri = ""
                if q_oneW_edges != []:
                    x,y = q_oneW_edges[0]
                    if u==x and y==v:
                        wri = f"{u} {v} {1}\n"
                        q_oneW_edges.pop(0)
                    else:
                        wri = f"{u} {v} {2}\n"
                else:
                    wri = f"{u} {v} {2}\n"
                # Writing data to a file
                file1.write(wri)



In [274]:
#### This is a good example
# n=8
# k=7

g = generate_graph([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8)

# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 2)
# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 3)
# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 4)
# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 5)
# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 6)
# generate_input([(1,2),(1,3), (2,3), (5,6),(5,7), (6,7)], 8, 7)


assert(g.maxConnectedOnes(2) == 2)
assert(g.maxConnectedOnes(3) == 4)
assert(g.maxConnectedOnes(4) == 5)
assert(g.maxConnectedOnes(5) == 6)
assert(g.maxConnectedOnes(6) == 8)
assert(g.maxConnectedOnes(7) == 10)


# ##### GREAT NOW THIS IS WRONG.....
S1 = generate_graph([(0,3),(1,2)], 4)
generate_input([(0,3),(1,2)], 4, 2)
assert(S1.maxConnectedOnes(2) == 3)
assert(S1.maxConnectedOnes(3) == 4)


-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
0->4, w(uv) = 2
0->5, w(uv) = 2
5->6, w(uv) = 1
5->7, w(uv) = 1
[2, 2]
k=3, blocklen=2
 ANS:4, new_k: 0
k=0, blocklen=2
 ANS:4, new_k: 0
-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
0->4, w(uv) = 2
0->5, w(uv) = 2
5->6, w(uv) = 1
5->7, w(uv) = 1
[2, 2]
k=4, blocklen=2
 ANS:4, new_k: 1
k=1, blocklen=2
 ANS:5, new_k: 0
-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
0->4, w(uv) = 2
0->5, w(uv) = 2
5->6, w(uv) = 1
5->7, w(uv) = 1
[2, 2]
k=5, blocklen=2
 ANS:4, new_k: 2
k=2, blocklen=2
 ANS:6, new_k: 0
-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
0->4, w(uv) = 2
0->5, w(uv) = 2
5->6, w(uv) = 1
5->7, w(uv) = 1
[2, 2]
k=6, blocklen=2
 ANS:4, new_k: 3
k=3, blocklen=2
 ANS:8, new_k: 0
-1->0, w(uv) = 0
0->1, w(uv) = 2
1->2, w(uv) = 1
1->3, w(uv) = 1
0->4, w(uv) = 2
0->5, w(uv) = 2
5->6, w(uv) = 1
5->7, w(uv) = 

In [125]:
g = Graph(9)

# Making the above shown graph
g.add_edge(0, 1, 1)
g.add_edge(0, 7, 2)
g.add_edge(1, 2, 2)
g.add_edge(1, 7, 2)
g.add_edge(2, 3, 2)
g.add_edge(2, 8, 2)
g.add_edge(2, 5, 2)
g.add_edge(3, 4, 2)
g.add_edge(3, 5, 1)
g.add_edge(4, 5, 2)
g.add_edge(5, 6, 1)
g.add_edge(6, 7, 1)
g.add_edge(6, 8, 2)
g.add_edge(7, 8, 2)

In [167]:
G3 = Graph(6)
G3.add_edge(0,1, 1)
G3.add_edge(0,2, 1)
G3.add_edge(0,3, 1)
G3.add_edge(0,4, 1)
G3.add_edge(0,5, 1)
G3.add_edge(5,1, 1)
G3.add_edge(1,2, 1)
G3.add_edge(2,3, 1)
G3.add_edge(3,4, 1)
G3.add_edge(4,5, 1)
G3.add_edge(1,4, 1)
G3.maxConnectedOnes(5)


-1->0, w(uv) = 0
0->1, w(uv) = 1
0->2, w(uv) = 1
0->3, w(uv) = 1
0->4, w(uv) = 1
0->5, w(uv) = 1
[0, 1, 1, 1, 1, 1]


In [129]:
G32 = Graph(6)
G32.add_edge(0,1, 2)
G32.add_edge(0,2, 2)
G32.add_edge(0,3, 2)
G32.add_edge(0,4, 1)
G32.add_edge(0,5, 1)
G32.add_edge(5,1, 2)
G32.add_edge(1,2, 2)
G32.add_edge(2,3, 2)
G32.add_edge(3,4, 2)
G32.add_edge(4,5, 1)
G32.add_edge(1,4, 1)

assert(G32.maxConnectedOnes(5)==3)
assert(G3.maxConnectedOnes(3)==3)
assert(G3.maxConnectedOnes(2)==2)

In [132]:
g = generate_graph([(1,2), (1,3), (6,7)], 9)
assert(g.maxConnectedOnes(8)==2)
assert(g.maxConnectedOnes(2)==2)


In [160]:
generate_input([(1,2),(1,3),(1,5), (3,7), (4,7), (4,20), (4,100), (4,200), (4, 300), (5,6),(5,7), (6,7)], 345, 11)


In [134]:
g = generate_graph([(1,2), (1,3), (6,7)], 500)
assert(g.maxConnectedOnes(499)==2)


In [136]:
g = generate_graph([(1,2),(1,4), (2,3), (3,4)], 500)
assert(g.maxConnectedOnes(499)==3)


In [140]:
g = generate_graph([(1,2),(1,3),(1,5), (3,7), (4,7), (4,20), (4,100), (4,200), (4, 300), (5,6),(5,7), (6,7)], 500)
assert(g.maxConnectedOnes(499)==10)


In [162]:
generate_input([
    (1,2), (1,3), (2,3), (20, 30), (20, 50), (30, 50), (30, 55), (30, 91),(50, 55), (60,70), (80, 90),
    (91, 92), (91, 93), (91, 94), (91, 95), (91, 96), (91, 97)
], 120, 11)


In [ ]:
# Driver program to test methods of the graph class
if __name__ == "__main__":
    # Create the graph given in the above figure
    V = 9
    g = Graph(V)

    # Making the above shown graph
    g.add_edge(0, 1, 4)
    g.add_edge(0, 7, 8)
    g.add_edge(1, 2, 8)
    g.add_edge(1, 7, 11)
    g.add_edge(2, 3, 7)
    g.add_edge(2, 8, 2)
    g.add_edge(2, 5, 4)
    g.add_edge(3, 4, 9)
    g.add_edge(3, 5, 14)
    g.add_edge(4, 5, 10)
    g.add_edge(5, 6, 2)
    g.add_edge(6, 7, 1)
    g.add_edge(6, 8, 6)
    g.add_edge(7, 8, 7)

    g.prim_mst()
